In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
from random import randint

class Crime:
    '''Формирование таблиц из XML-файла'''
    def __init__(self, xml="allIndicators.xml"):
        self.data = ET.parse(xml).getroot()
        
    def get_values(self, values):
        '''{'Тверская область': [{(2020, 4): 6946}]}'''
        v = [v.text for v in values]
        return {v[2]: [ {(int(v[3].split('-')[0]), int(v[3].split('-')[1])): int(v[0])}]}
        
    def get_row(self, rows):
        # {'Тверская область': [{(2020, 4): 6946}, (2020, 3): 1234}, ...]}
        result = {}
        for row in rows:            
            key = list(self.get_values(row))[0]
            value = list(self.get_values(row).values())[0][0]
            v = list(value.keys())[0][0]
            if v not in {2009, 2010, 2011}: # берем только с 2012 года
                if key in result:
                    result[key].append(value)
                else:
                    result[key] = [value]
        for key, value in result.items():
            result[key] = sorted(value, key=lambda k: max(k.keys()))
        return result
    
    def get_full_dict(self):
        '''{'Всего .. ний': {'Тв.. ..сть': [{(2009, 12): 33888}, {(2010, 1): 2879}, ...'''
        result = {}
        for crime in self.data:
            result[crime.attrib['name']] = self.get_row(crime)
        return result
    
    def get_frame(self, crime):
        '''Словарь DataFrame'''
        regions = {}            
        for region, value in crime.items():
            columns = [list(item.keys())[0] for item in value]
            rows = [list(item.values())[0] for item in value]
            regions[region] = rows
        return pd.DataFrame.from_dict(regions, orient='index', columns=columns)
    
    def get_files(self):
        '''Сохраняем в Excel-файлы'''
        crimes = self.get_full_dict()
        for crime, value in crimes.items():
            try:
                df = self.get_frame(value)
                with pd.ExcelWriter(f'result/{crime.replace(",", "")}.xlsx') as writer:
                    df.to_excel(writer, sheet_name="data")
            except ValueError:
                print(f'Problem: {crime}')
                    

crime = Crime()
crime.get_files()

Problem: Зарегистрировано преступлений по ч. 4, ст. 111 УК РФ - умышленное причинение тяжкого вреда здоровью, повлекшее по неосторожности смерть потерпевшего
